# Cleaning demographics data collected from Wikipedia

The following steps will result in clean demographic data as a csv output file:
 1. Make state column as index, rename index to 'State'
 2. Get rid of all 'Rank' or rank-like columns
 3. Add dataframe name to column names.
 4. Merge dataframes

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
all_indicators = pd.read_excel("India_Demographics.xlsx", sheetname=None)

In [3]:
keys = list(all_indicators.keys())

## 1. Make state column as index, rename index to 'State'

In [4]:
for index, data in enumerate(all_indicators.values()):
    columns = Series(data.columns.values)
    index_column = columns[columns.str.lower().str.find('state') >= 0]
    data.set_index(index_column.iloc[0], inplace=True)
    data.index.name='State'
    data.name=keys[index]

## 2. Get rid of all 'Rank' or rank-like columns

In [5]:
for data in all_indicators.values():
    columns = Series(data.columns.values)
    rank_columns = columns[columns.str.lower().str.contains('rank').fillna(True)].tolist()
    data.drop(rank_columns, axis=1, inplace=True)

## 3. Add dataframe name to column names

In [6]:
for index, data in enumerate(all_indicators.values()):
    data.columns = Series(data.columns.values).apply(lambda x: keys[index] + '-' + x)

## 4. Merge dataframes

In [7]:
for i in range(len(keys)):
    #print(i)
    all_indicators[keys[0]]=all_indicators[keys[0]].merge(all_indicators[keys[i]], how='outer',left_index=True,right_index=True)
    
data = all_indicators[keys[0]]   

## 5. Replace index (states) with values from original data

In [8]:
result_states = ['A & N Islands', 'Andhra Pradesh', 'Arunachal Pradesh', 'Assam',
       'Bihar', 'Chandigarh', 'Chhattisgarh', 'D & N Haveli',
       'Daman & Diu', 'Delhi (Ut)', 'Goa', 'Gujarat', 'Haryana',
       'Himachal Pradesh', 'Jammu & Kashmir', 'Jharkhand', 'Karnataka',
       'Kerala', 'Lakshadweep', 'Madhya Pradesh', 'Maharashtra', 'Manipur',
       'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Puducherry',
       'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Tripura',
       'Uttar Pradesh', 'Uttarakhand', 'West Bengal']

In [9]:
import re
state_matches = {}
#A & N Islands
state_matches[result_states[0]] = 'A.*N.*'
state_matches[result_states[1]] = 'Andhra.*|Tel.*'
state_matches[result_states[2]] = 'Arunachal.*'
state_matches[result_states[3]] = 'Assam'
state_matches[result_states[4]] = 'Bihar'
state_matches[result_states[5]] = 'Chandigarh'
state_matches[result_states[6]] = 'Chhattisgarh'
state_matches[result_states[7]] = 'D.*N.*'
state_matches[result_states[8]] = 'D.*D.*'
state_matches[result_states[9]] = 'Delhi'
state_matches[result_states[10]] = 'Goa'
state_matches[result_states[11]] = 'Gujarat'
state_matches[result_states[12]] = 'Haryana'
state_matches[result_states[13]] = 'Himachal.*'
state_matches[result_states[14]] = 'J.*K.*'
state_matches[result_states[15]] = 'Jharkhand'
state_matches[result_states[16]] = 'Karnataka'
state_matches[result_states[17]] = 'Kerala'
state_matches[result_states[18]] = 'Lakshadweep'
state_matches[result_states[19]] = 'Madhya.*'
state_matches[result_states[20]] = 'Maharashtra.*'
state_matches[result_states[21]] = 'Manipur.*'
state_matches[result_states[22]] = 'Meghalaya.*'
state_matches[result_states[23]] = 'Mizoram.*'
state_matches[result_states[24]] = 'Nagaland.*'
state_matches[result_states[25]] = 'Odisha.*'
state_matches[result_states[26]] = 'Pudu.*'
state_matches[result_states[27]] = 'Punjab.*'
state_matches[result_states[28]] = 'Raja.*'
state_matches[result_states[29]] = 'Sikkim.*'
state_matches[result_states[30]] = 'Tamil.*'
state_matches[result_states[31]] = 'Tripura.*'
state_matches[result_states[32]] = 'Uttar P.*'
state_matches[result_states[33]] = 'Uttarakhand.*'
state_matches[result_states[34]] = 'West.*'

In [10]:
index_values = data.index
count = 0
for key, match in state_matches.items():
    data.index = data.index.str.replace(match,key)
    #count += len(matched_values)
    #print (count, matched_values)

print (data.index)

Index(['A & N Islands', 'All  India  Average', 'All India', 'A & N Islands',
       'A & N Islands', 'A & N Islands', 'Andhra Pradesh', 'Andhra Pradesh',
       'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'D & N Haveli', 'D & N Haveli', 'D & N Haveli',
       'Daman & Diu', 'Daman & Diu', 'Delhi (Ut)', 'Goa', 'Gujarat', 'Haryana',
       'Himachal Pradesh', 'INDIA', 'India', 'India  average',
       'Jammu & Kashmir', 'Jammu & Kashmir', 'Jharkhand', 'Karnataka',
       'Kerala', 'Lakshadweep', 'Madhya Pradesh', 'Maharashtra', 'Manipur',
       'Meghalaya', 'Mizoram', 'Nagaland', 'National average', 'Odisha',
       'Puducherry', 'Puducherry', 'Punjab', 'Punjab', 'Rajasthan', 'Sikkim',
       'Tamil Nadu', 'Andhra Pradesh', 'Andhra Pradesh', 'Total', 'Tripura',
       'Uttar Pradesh', 'Uttarakhand', 'West Bengal', 'Whole  INDIA'],
      dtype='object', name='State')


## 6. Combine rows with the same index

In [11]:
#def aggreg
#data.groupby(data.index).agg()
data = data.groupby(data.index).agg(lambda x: x.dropna().tolist())


In [12]:
def consolidate_list(x):
    if all(isinstance(i, int) for i in x):
        return np.mean(x)
    else:
        return x[0]

data = data.applymap(consolidate_list)
data = data.dropna(axis = 0, thresh = 18)

# Remove repeated columns
data = data.drop(data.columns[data.columns.str.contains('_y')], axis=1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## 7. Convert string elements to float

In [13]:
import re
def check_and_convert_to_float(x):
    if isinstance(x, str):
        result = re.search(r'\d+', x.replace(',', ''))
        if result:
            return result.group()
    else:
        return float(x)
data = data.applymap(check_and_convert_to_float)

## 8. Save to csv!

In [14]:
data.to_csv('India_demographics.csv')